# Curate DataFrames and AnnDatas

Curating a dataset with LaminDB means three things:

1. Validate that the dataset matches a desired schema
2. In case the dataset doesn't validate, standardize it, e.g., by fixing typos or mapping synonyms
3. Annotate the dataset by linking it against metadata entities so that it becomes queryable

## Curate a DataFrame

In [ ]:
# pip install 'lamindb[bionty]'
!lamin init --storage ./test-curate --modules bionty

Let's start with a DataFrame that we'd like to validate.

In [ ]:
import lamindb as ln
import bionty as bt
import pandas as pd


df = pd.DataFrame(
    {
        "cell_medium": pd.Categorical(["DMSO", "IFNG", "DMSO"]),
        "temperature": [37.2, 36.3, 38.2],
        "cell_type": pd.Categorical(
            [
                "cerebral pyramidal neuron",
                "astrocytic glia",
                "oligodendrocyte",
            ]
        ),
        "assay_ontology_id": pd.Categorical(
            ["EFO:0008913", "EFO:0008913", "EFO:0008913"]
        ),
        "donor": ["D0001", "D0002", "D0003"],
    },
    index=["obs1", "obs2", "obs3"],
)
df

Define a schema to validate this dataset.

In [ ]:
schema = ln.Schema(
    name="My example schema",
    features=[
        ln.Feature(name="cell_medium", dtype=ln.ULabel).save(),
        ln.Feature(name="temperature", dtype=float).save(),
        ln.Feature(name="cell_type", dtype=bt.CellType).save(),
        ln.Feature(
            name="assay_ontology_id", dtype=bt.ExperimentalFactor.ontology_id
        ).save(),
        ln.Feature(name="donor", dtype=str).save(),
    ],
).save()
# look at the schema
schema.features.df()

In [ ]:
curator = ln.curators.DataFrameCurator(df, schema)

The {meth}`~lamindb.curators.Curator.validate` method checks our data against the defined criteria. It identifies which values are already validated (exist in our registries) and which are potentially problematic (do not yet exist in our registries).

In [ ]:
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(error)

In [ ]:
# check the non-validated terms
curator.cat.non_validated

For `cell_type`, we saw that "cerebral pyramidal neuron", "astrocytic glia" are not validated.

First, let's standardize synonym "astrocytic glia" as suggested

In [ ]:
curator.cat.standardize("cell_type")

In [ ]:
# now we have only one non-validated cell type left
curator.cat.non_validated

For "cerebral pyramidal neuron", let's understand which cell type in the public ontology might be the actual match.

In [ ]:
# to check the correct spelling of categories, pass `public=True` to get a lookup object from public ontologies
# use `lookup = curator.cat.lookup()` to get a lookup object of existing records in your instance
lookup = curator.cat.lookup(public=True)
lookup

In [ ]:
# here is an example for the "cell_type" column
cell_types = lookup["cell_type"]
cell_types.cerebral_cortex_pyramidal_neuron

In [ ]:
# fix the cell type
df.cell_type = df.cell_type.replace(
    {"cerebral pyramidal neuron": cell_types.cerebral_cortex_pyramidal_neuron.name}
)

For donor, we want to add the new donors: "D0001", "D0002", "D0003"

In [ ]:
# this adds donors that were _not_ validated
curator.cat.add_new_from("cell_medium")

In [ ]:
# validate again
curator.validate()

Save a curated artifact.

In [ ]:
artifact = curator.save_artifact(key="my_datasets/my_curated_dataset.parquet")

In [ ]:
artifact.describe()

## Curate an AnnData

Here we additionally specify which `var_index` to validate against.

In [ ]:
import anndata as ad

X = pd.DataFrame(
    {
        "ENSG00000081059": [1, 2, 3],
        "ENSG00000276977": [4, 5, 6],
        "ENSG00000198851": [7, 8, 9],
        "ENSG00000010610": [10, 11, 12],
        "ENSG00000153563": [13, 14, 15],
        "ENSGcorrupted": [16, 17, 18],
    },
    index=df.index,  # because we already curated the dataframe above, it will validate
)
adata = ad.AnnData(X=X, obs=df)
adata

In [ ]:
# define var schema
var_schema = ln.Schema(
    name="my_var_schema",
    itype=bt.Gene.ensembl_gene_id,
    dtype=int,
).save()

# define composite schema
anndata_schema = ln.Schema(
    name="small_dataset1_anndata_schema",
    otype="AnnData",
    components={"obs": schema, "var": var_schema},
).save()

In [ ]:
var_schema.itype

In [ ]:
curator = ln.curators.AnnDataCurator(adata, anndata_schema)
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(error)

Subset the `AnnData` to validated genes only:

In [ ]:
adata_validated = adata[:, ~adata.var.index.isin(["ENSGcorrupted"])].copy()

Now let's validate the subsetted object:

In [ ]:
curator = ln.curators.AnnDataCurator(adata_validated, anndata_schema)
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(error)

The validated object can be subsequently saved as an {class}`~lamindb.Artifact`:

In [ ]:
artifact = curator.save_artifact(key="my_datasets/my_curated_anndata.h5ad")

Saved artifact has been annotated with validated features and labels:

In [ ]:
artifact.describe()

We've walked through the process of validating, standardizing, and annotating datasets going through these key steps:

1. Defining validation criteria
2. Validating data against existing registries
3. Adding new validated entries to registries
4. Annotating artifacts with validated metadata

By following these steps, you can ensure your data is standardized and well-curated.

If you have datasets that aren't DataFrame-like or AnnData-like, read: {doc}`curate-any`.

In [ ]:
!rm -rf ./test-curate
!lamin delete --force test-curate